In [294]:
import pandas as pd
import telebot
from time import sleep
import pickle
import gc

In [295]:
from surprise import Dataset, SVD,  Reader, BaselineOnly, accuracy
from surprise import KNNBasic
from surprise.model_selection import cross_validate
from collections import defaultdict
from surprise.model_selection import KFold

In [296]:
with open('item_ids_dict', 'rb') as f:
  item_ids_dict = pickle.load(f)
  f.close()
with open('item_ids_dict_rev', 'rb') as f:
  item_ids_dict_rev = pickle.load(f)
  f.close()


In [297]:
bot = telebot.TeleBot("6289827643:AAFB8Ju5E1Jw7M2GakLTb37HoBhtNUD5S3U")

In [298]:
@bot.message_handler(commands=['start'])

def start(message):
    bot.send_message(message.chat.id, "Hi and thanks for checking out my perfume recommender bot! Use /help for a quick start.")


In [299]:

@bot.message_handler(commands=['help'])

def help(message):
    bot.send_message(message.chat.id, "<b>How it works:</b>", parse_mode="html")
    sleep(1)
    bot.send_message(message.chat.id, "1) We give you a list of perfumes")
    sleep(1)
    bot.send_message(message.chat.id, "2) You rate at least 3 perfumes from the list (the more, the better)")
    sleep(1)
    bot.send_message(message.chat.id, "3) With the use of black magic, we recommend you 10 perfumes that you will probably enjoy")
    sleep(1)
    bot.send_message(message.chat.id, "The recommendations are made based on user data from fragnancenet.com. Also, keep in mind that recommendations are tailored for women.")
    sleep(1)
    bot.send_message(message.chat.id, "The current list of supported perfumes is quite large. However, some perfumes from the site are missing, since there are not enough user ratings.")
    sleep(1)
    bot.send_message(message.chat.id, "To enhance your experience, please follow this link: https://www.fragrancenet.com/fragrances?f=1f!3D")
    sleep(1)
    bot.send_message(message.chat.id, "From the filtered list, find the perfumes that you already own (at least 3 perfumes). If you don't own three perfumes, do not worry - we will recommend you the prefumes our users enjoy the most.")
    sleep(1)
    bot.send_message(message.chat.id, "Once you have found all of the items, use /getrecs command to get recommendations.")
    

In [300]:
def decode(top_n, decodefrom):
    decoded = defaultdict(list)
    for user in top_n.keys():
        for i in top_n[user]:
            decoded[user].append(decodefrom[i]) 
    return decoded

In [301]:
@bot.message_handler(commands=['getrecs'])

def getrecs(message):
    sent = bot.send_message(message.chat.id, "Please enter the perfume-rating data. Make to format it correctly as Name,Rating")
    sleep(1)
    bot.send_message(message.chat.id, "Here is an example: Cool Water, 4, Versace Bright Crystal Absolu, 5, Eternity Perfume, 3")                       
    bot.register_next_step_handler(sent, perfumes)
    
def perfumes(message):
    
    u_items = []
    u_ratings = []
    rev_items = []

    ins = message.text.split(",")

    try:
        for i in range(0, len(ins)):

            if i%2 == 0:
                try:
                    rev_items.append(item_ids_dict_rev[ins[i].strip().lower()])
                    u_items.append(ins[i].strip().lower())
                except Exception:
                    bot.send_message(message.chat.id, f"{ins[i].strip()} is not in the supported list. Please try another item.")

            else:     
                try:
                    u_ratings.append(int(ins[i].strip()))
                except Exception:
                    bot.send_message(message.chat.id, 'Invalid input, please try again.')

        try:
            assert len(u_items) == len(u_ratings)
            bot.send_message(message.chat.id, 'You have a great taste!')
            bot.send_message(message.chat.id, 'Retrieving your personal recommendations, hold tight!')
        except Exception:
            bot.send_message(message.chat.id, "You are missing a name or a rating - please check your input and try again.")
        
        maindata = pd.read_csv(r'E:\parsing project\atleast20items.csv')
        topnlist = maindata.groupby('item_id')['author_id'].count().sort_values(ascending=False).index
        lessthan3 = {10000000:[i for i in topnlist if i not in rev_items][:10]}
        

        userdict = pd.DataFrame({'author_id':[10000000]*len(u_items), 'item_id':rev_items, 'num_rating':u_ratings})
        
        if len(u_items) >= 3:
        
        
            full_data = pd.concat([maindata, userdict])
            reader = Reader(line_format="user item rating", rating_scale=(1,5))
            data = Dataset.load_from_df(full_data, reader = reader)
            full_train = data.build_full_trainset()
            #bsl_options = {"method": "als", "n_epochs": 10, "reg_u": 10, "reg_i": 5}
            #algo = BaselineOnly(bsl_options=bsl_options)
            sim_options = {'name': 'cosine',
                   'user_based': False  
                   }
            algo = KNNBasic(sim_options=sim_options, k = 5)
            algo.fit(full_train)
            bot.send_message(message.chat.id, "Building testest...")
            pred_this = [(10000000, i) for i in list(set(item_ids_dict.keys()).difference(rev_items))]
            #full_test = full_train.build_anti_testset()
            bot.send_message(message.chat.id, "Success.")
            sleep(1)
            bot.send_message(message.chat.id, "Predicting...")
            preds = [algo.predict(uid, iid) for uid,iid in pred_this]
            bot.send_message(message.chat.id, "Success.")
            user_preds = [preds[i] for i in range(len(preds))]
            bot.send_message(message.chat.id, "Decoding predictions...")
            def get_top_n(predictions, n, items_onlyf = False):
            
            
                # First map the predictions to each user.
                top_n = defaultdict(list)
                for uid, iid, true_r, est, _ in predictions:
                    top_n[uid].append((iid, est))
                # Then sort the predictions for each user and retrieve the k highest ones.
                for uid, user_ratings in top_n.items():
                    user_ratings.sort(key=lambda x: x[1], reverse=True)
                    top_n[uid] = user_ratings[:n]
                if items_onlyf == True:
                
                    items_only = defaultdict(list)
                    for i in top_n.keys():
                        lst = [item for item, val in top_n[i]]
                        items_only[i] = lst
                return items_only if items_onlyf == True else top_n    
            #top items that user has not rated
            recommended = get_top_n(user_preds, n=10, items_onlyf= True)
            #decode predictions 

            final_preds = decode(recommended, item_ids_dict)
            #bot.send_message(message.chat.id, preds)
            
            for_links = []
            for i in final_preds[10000000]:
                for_links.append('https://www.fragrancenet.com/search/' + i.replace(' ', '%20'))
            bot.send_message(message.chat.id, 'Here are top 10 perfumes you might enjoy:')
            
            for i, j in zip(final_preds[10000000], for_links):
                bot.send_message(message.chat.id, f'{i}:')
                bot.send_message(message.chat.id, j)
                sleep(1)
            bot.send_message(message.chat.id, 'Thank you for using our service. Come again soon!')
            
            del full_train, preds
            gc.collect()
        
        else:
            bot.send_message(message.chat.id, 'Since you selected less than 3 items, here are top 10 items our user enjoy the most:')

            s = decode(lessthan3, item_ids_dict)


            for i in s[10000000]:
                bot.send_message(message.chat.id, f'{i}:')
                bot.send_message(message.chat.id, 'https://www.fragrancenet.com/search/' + i.replace(' ', '%20'))
                sleep(1)
                
        
    except Exception:
        bot.send_message(message.chat.id, 'Invalid input, please try again.')



    

          
        
    

In [302]:
bot.polling(none_stop=True)



Computing the cosine similarity matrix...
Done computing similarity matrix.


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))